# Calibration & Reduction

Reduction details and method



### Single-dish data calibration and reduction

Generally, the calibration of ALMA single-dish data requires essentially
only two steps - the application of the $T_{sys}$ calibration, and the
application of the "sky" calibration (i.e. OFF). With just these steps,
the product is $T_A^*$ in units of Kelvin. While the
steps are described in detail here, an example of the full single-dish
calibration process can be found in the [M100 Band 3 Single Dish
CASAguide](https://casaguides.nrao.edu/index.php/M100_Band3_SingleDish_4.3).

In the following description, we refer to data in the ALMA-native format
of ASDM (ALMA Science Data Model) with a variable name *asdm*, while we
refer to the CASA-native format of MeasurementSet with a variable name
*sd_ms*. In the case of single-dish data, the MeasurementSet data are
not formally *visibilities*, since they are simply auto-correlations.

1.  **importasdm**: Import the ASDM
2.  **listobs**&nbsp;- List observation parameters
3.  **flagdata** and **flagcmd** - A priori flagging
4.  **gencal**,&nbsp;**applycal**, and **sdcal**- Generate and apply the
    $T_{sys}$ and sky calibration tables
5.  **flagdata** - Do initial flagging
6.  **applycal** -&nbsp; Calibrate the data into Kelvins
7.  **sdbaseline** - Subtract the baseline

**1. Import of the ASDM:&nbsp;*ImportASDM***

Import of ASDM data and conversion into the MeasurementSet format is
achieved via the **importasdm** task. The task settings are a little
different to those used when importing interferometer ASDMs. For
instance,&nbsp;*bdfflags=False*&nbsp;will de-select application of the
BDF format flags, which are of no consequence in the normal reduction
process, and&nbsp;*with_pointing_correction=True*&nbsp;will directly
apply the *measured*&nbsp;pointing direction (in Azimuth and Elevation)
to the data, rather than the *commanded*&nbsp;pointing positions. For
example:

importasdm(asdm, asis='Antenna Station Receiver Source CalAtmosphere
CalWVR CorrelatorMode SBSummary', bdfflags=False,
process_caldevice=False, with_pointing_correction=True)

&nbsp;

**2. List observations parameters:&nbsp;*listobs*&nbsp;**

**listobs** works for single dish observations in the same way as for
interferometry, and will detail the calibration scans.&nbsp; It
identifies pointing, $T_{sys}$, and target scans, the science
target and OFF scans and their cadence, the correlator frequencies and
configuration, and the antennas used in the observations. Unique to the
single-dish data is the \#OFF_SOURCE intent. This task is used to
identify which spectral windows are used for $T_{sys}$
observations.&nbsp; This information is critical for those who wish to
build their own calibrations.

listobs(sd_ms)

produces feedback in the logger (or optionally, to a file) from which we
can determine which spectral windows are $T_{sys}$ observations and which are
science observations.

Spectral Windows: (25 unique spectral windows and 2 unique polarization
setups)  
SpwID Name \#Chans Frame Ch0(MHz) ChanWid(kHz) TotBW(kHz) CtrFreq(MHz)
BBC Num Corrs  

&nbsp; 0 &nbsp;BB_1\#SQLD &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
&nbsp; &nbsp; &nbsp; 1 &nbsp; TOPO&nbsp; 195994.575 &nbsp; 2000000.000
&nbsp; 2000000.0 195994.5750&nbsp; &nbsp; &nbsp; &nbsp; 1&nbsp; XX&nbsp;
YY  
&nbsp; 1 &nbsp;BB_2\#SQLD &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
&nbsp; &nbsp; &nbsp; 1 &nbsp; TOPO&nbsp; 197932.075 &nbsp; 2000000.000
&nbsp; 2000000.0 197932.0750&nbsp; &nbsp; &nbsp; &nbsp; 2&nbsp; XX&nbsp;
YY  
&nbsp; 2 &nbsp;BB_3\#SQLD &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
&nbsp; &nbsp; &nbsp; 1 &nbsp; TOPO&nbsp; 207994.575 &nbsp; 2000000.000
&nbsp; 2000000.0 207994.5750&nbsp; &nbsp; &nbsp; &nbsp; 3&nbsp; XX&nbsp;
YY  
&nbsp; 3 &nbsp;BB_4\#SQLD &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
&nbsp; &nbsp; &nbsp; 1 &nbsp; TOPO&nbsp; 209994.575 &nbsp; 2000000.000
&nbsp; 2000000.0 209994.5750&nbsp; &nbsp; &nbsp; &nbsp; 4&nbsp; XX&nbsp;
YY  
&nbsp; 4 &nbsp;WVR\#NOMINAL &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
&nbsp; &nbsp; 4 &nbsp; TOPO&nbsp; 184550.000 &nbsp; 1500000.000 &nbsp;
7500000.0 187550.0000&nbsp; &nbsp; &nbsp; &nbsp; 0&nbsp; XX  
&nbsp; 5 &nbsp;X436890472\#ALMA_RB_05\#BB_1\#SW-01\#FULL_RES &nbsp;
&nbsp; 128 &nbsp; TOPO&nbsp; 196986.763&nbsp; &nbsp; -15625.000 &nbsp;
2000000.0 195994.5750&nbsp; &nbsp; &nbsp; &nbsp; 1&nbsp; XX&nbsp; YY  
&nbsp; 6 &nbsp;X436890472\#ALMA_RB_05\#BB_1\#SW-01\#CH_AVG &nbsp; &nbsp;
&nbsp; &nbsp; 1 &nbsp; TOPO&nbsp; 195978.950 &nbsp; 1796875.000 &nbsp;
1796875.0 195978.9500&nbsp; &nbsp; &nbsp; &nbsp; 1&nbsp; XX&nbsp; YY  
&nbsp; 7 &nbsp;X436890472\#ALMA_RB_05\#BB_2\#SW-01\#FULL_RES &nbsp;
&nbsp; 128 &nbsp; TOPO&nbsp; 198924.263&nbsp; &nbsp; -15625.000 &nbsp;
2000000.0 197932.0750&nbsp; &nbsp; &nbsp; &nbsp; 2&nbsp; XX&nbsp; YY  
&nbsp; 8 &nbsp;X436890472\#ALMA_RB_05\#BB_2\#SW-01\#CH_AVG &nbsp; &nbsp;
&nbsp; &nbsp; 1 &nbsp; TOPO&nbsp; 197916.450 &nbsp; 1796875.000 &nbsp;
1796875.0 197916.4500&nbsp; &nbsp; &nbsp; &nbsp; 2&nbsp; XX&nbsp; YY  
&nbsp; 9 &nbsp;X436890472\#ALMA_RB_05\#BB_3\#SW-01\#FULL_RES &nbsp;
&nbsp; 128 &nbsp; TOPO&nbsp; 207002.388 &nbsp; &nbsp; 15625.000 &nbsp;
2000000.0 207994.5750&nbsp; &nbsp; &nbsp; &nbsp; 3&nbsp; XX&nbsp; YY  
&nbsp; 10 X436890472\#ALMA_RB_05\#BB_3\#SW-01\#CH_AVG &nbsp; &nbsp;
&nbsp; &nbsp; 1 &nbsp; TOPO&nbsp; 207978.950 &nbsp; 1796875.000 &nbsp;
1796875.0 207978.9500&nbsp; &nbsp; &nbsp; &nbsp; 3&nbsp; XX&nbsp; YY  
&nbsp; 11 X436890472\#ALMA_RB_05\#BB_4\#SW-01\#FULL_RES &nbsp; &nbsp;
128 &nbsp; TOPO&nbsp; 209002.388 &nbsp; &nbsp; 15625.000 &nbsp;
2000000.0 209994.5750&nbsp; &nbsp; &nbsp; &nbsp; 4&nbsp; XX&nbsp; YY  
&nbsp; 12 X436890472\#ALMA_RB_05\#BB_4\#SW-01\#CH_AVG &nbsp; &nbsp;
&nbsp; &nbsp; 1 &nbsp; TOPO&nbsp; 209978.950 &nbsp; 1796875.000 &nbsp;
1796875.0 209978.9500&nbsp; &nbsp; &nbsp; &nbsp; 4&nbsp; XX&nbsp; YY  
&nbsp; 13 BB_1\#SQLD &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
&nbsp; &nbsp; 1 &nbsp; TOPO&nbsp; 183375.638 &nbsp; 2000000.000 &nbsp;
2000000.0 183375.6378&nbsp; &nbsp; &nbsp; &nbsp; 1&nbsp; XX&nbsp; YY  
&nbsp; 14 BB_2\#SQLD &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
&nbsp; &nbsp; 1 &nbsp; TOPO&nbsp; 181427.463 &nbsp; 2000000.000 &nbsp;
2000000.0 181427.4627&nbsp; &nbsp; &nbsp; &nbsp; 2&nbsp; XX&nbsp; YY  
&nbsp; 15 BB_3\#SQLD &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
&nbsp; &nbsp; 1 &nbsp; TOPO&nbsp; 169374.840 &nbsp; 2000000.000 &nbsp;
2000000.0 169374.8404&nbsp; &nbsp; &nbsp; &nbsp; 3&nbsp; XX&nbsp; YY  
&nbsp; 16 BB_4\#SQLD &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
&nbsp; &nbsp; 1 &nbsp; TOPO&nbsp; 170917.638 &nbsp; 2000000.000 &nbsp;
2000000.0 170917.6378&nbsp; &nbsp; &nbsp; &nbsp; 4&nbsp; XX&nbsp; YY  
&nbsp; 17 X1857092512\#ALMA_RB_05\#BB_1\#SW-01\#FULL_RES &nbsp; 4096
&nbsp; TOPO&nbsp; 183162.808 &nbsp; &nbsp; &nbsp; 122.070&nbsp; &nbsp;
500000.0 183412.7471&nbsp; &nbsp; &nbsp; &nbsp; 1&nbsp; XX&nbsp; YY  
&nbsp; 18 X1857092512\#ALMA_RB_05\#BB_1\#SW-01\#CH_AVG&nbsp; &nbsp;
&nbsp; &nbsp; 1 &nbsp; TOPO&nbsp; 183412.686&nbsp; &nbsp;
500000.000&nbsp; &nbsp; 500000.0 183412.6861&nbsp; &nbsp; &nbsp; &nbsp;
1&nbsp; XX&nbsp; YY  
&nbsp; 19 X1857092512\#ALMA_RB_05\#BB_2\#SW-01\#FULL_RES &nbsp; 4096
&nbsp; TOPO&nbsp; 181177.524 &nbsp; &nbsp; &nbsp; 122.070&nbsp; &nbsp;
500000.0 181427.4627&nbsp; &nbsp; &nbsp; &nbsp; 2&nbsp; XX&nbsp; YY  
&nbsp; 20 X1857092512\#ALMA_RB_05\#BB_2\#SW-01\#CH_AVG&nbsp; &nbsp;
&nbsp; &nbsp; 1 &nbsp; TOPO&nbsp; 181427.402&nbsp; &nbsp;
500000.000&nbsp; &nbsp; 500000.0 181427.4017&nbsp; &nbsp; &nbsp; &nbsp;
2&nbsp; XX&nbsp; YY  
&nbsp; 21 X1857092512\#ALMA_RB_05\#BB_3\#SW-01\#FULL_RES &nbsp; 4096
&nbsp; TOPO&nbsp; 169587.670 &nbsp; -122.070&nbsp; &nbsp; 500000.0
169337.7310&nbsp; &nbsp; &nbsp; &nbsp; 3&nbsp; XX&nbsp; YY  
&nbsp; 22 X1857092512\#ALMA_RB_05\#BB_3\#SW-01\#CH_AVG&nbsp; &nbsp;
&nbsp; &nbsp; 1 &nbsp; TOPO&nbsp; 169337.670&nbsp; &nbsp;
500000.000&nbsp; &nbsp; 500000.0 169337.6700&nbsp; &nbsp; &nbsp; &nbsp;
3&nbsp; XX&nbsp; YY  
&nbsp; 23 X1857092512\#ALMA_RB_05\#BB_4\#SW-01\#FULL_RES &nbsp; 4096
&nbsp; TOPO&nbsp; 171158.788 &nbsp; -122.070&nbsp; &nbsp; 500000.0
170908.8487&nbsp; &nbsp; &nbsp; &nbsp; 4&nbsp; XX&nbsp; YY  
&nbsp; 24 X1857092512\#ALMA_RB_05\#BB_4\#SW-01\#CH_AVG&nbsp; &nbsp;
&nbsp; &nbsp; 1 &nbsp; TOPO&nbsp; 170908.788&nbsp; &nbsp;
500000.000&nbsp; &nbsp; 500000.0 170908.7877&nbsp; &nbsp; &nbsp; &nbsp;
4&nbsp; XX&nbsp; YY

From this output, we see the science spectral windows are 17, 19, 21 and
23, and have 4096 channels, while the $T_{sys}$ spectral windows at 5, 7, 9 and
11 have 128 channels.

&nbsp;

**3. A priori flagging:&nbsp;*flagcmd/flagdata***

**flagcmd** works the same way on single-dish data as for
interferometry. In this case, invoking it here applies flagging, by
default, from the FLAG_CMD file within the MeasurementSet.

flagcmd(vis = 'uid\_\_\_A002_Xb978c3_X5c4b.ms', inpmode =
'table',&nbsp;useapplied = True,&nbsp;action = 'apply')

**flagdata** is used at this point to remove problematic data.
Conventionally, 5% of the edges of the bands are removed, as these parts
of the band are significantly and detrimentally affected by the
low-sensitivity edges of the filter passband. In principle, they can be
retained in the cases where spectral lines of interest fall in that
area, though the sensitivity losses are significant.

Users should examine their spectra using **plotms**, and ensure any
atmospheric lines are properly accounted for. This is particularly true
for Band 5 which has a strong atmospheric absorption line at $\sim$183 GHz.
There is no real way to remove the signature of the atmospheric lines in
position-switched data, since the elevations of the ON (science target)
and OFF (sky-calibration position) are almost always different, and
therefore have different air masses. The most effective approach in this
case is to complete the normal calibrations as described here, then
apply a judiciously-selected bandpass correction polynomial and spectral
window channel range, as described by the **sdbaseline** step below.

flagdata(vis=vis, mode='manual',
spw='17:0\~119;3960\~4079,19:0\~119;3960\~4079,21:0\~119;3960\~4079,23:0\~119;3960\~4079',
action='apply', flagbackup=True)

Both steps **flagcmd** and **flagdata** are generally useful, but care
should be taken in case the emission lines of interest are being
inadvertantly flagged out.

&nbsp;

**4. Generation of the **$T_{sys}$ and $T_{sky}$
calibration tables:&nbsp;*gencal, sdcal and applycal*

There are two ways to proceed in CASA when computing and applying
calibration tables for single dish observations.

1.  Build the $T_{sys}$ calibration tables with
    **gencal**, build the sky calibration tables with **sdcal,** and
    apply them with **applycal**
2.  Build and apply both the $T_{sys}$ and sky calibration tables
    with **sdcal**

The second option is faster, but users familar with the **gencal** and
**applycal** tasks may prefer the first option.

In either case, the mapping between the $T_{sys}$ scans and science scans must be
determined either by examination of the output of **listobs**, or by
running the **sdcal** and specifying the method to be used to obtain the
OFF position. Usually ALMA will take&nbsp;position-switched observations
via *mode=*'*ps*', though other alternatives exist which do not need any
OFF positions to be explicitly observed. The&nbsp;OFF can be obtained
from the source data itself via *mode=*'*otfraster*' or *mode=*'*off*'.

&nbsp;

In the first of the two cases mentioned above (having identified the
target spectral windows as 17,19,21 and 23, and using a target
identified by the variable name, "source") :

gencal(vis = sd_ms,&nbsp;caltable = sd_ms+'.tsys',&nbsp;caltype =
'tsys')

sdcal(infile = sd_ms,&nbsp;outfile = sd_ms+'.sky',&nbsp;calmode =
'ps')  
  
from recipes.almahelpers import tsysspwmap  
tsysmap = tsysspwmap(vis = sd_ms, tsystable = sd_ms+'.tsys', trim =
False)  
  
applycal(vis = sd_ms,&nbsp;applymode = 'calflagstrict',&nbsp;spw =
'17,19,21,23',&nbsp;field = source,&nbsp;gaintable =
\[sd_ms+'.tsys',&nbsp;sd_ms+'.sky'\],&nbsp;gainfield = \['nearest',
source\],&nbsp;spwmap = tsysmap)

&nbsp;

In the second case:

sdcal(infile=sd_ms, calmode='ps,tsys,apply')

Note that we didn't specify the $T_{sys}$ spectral windows in the call to
**sdcal.&nbsp;** For ALMA single-dish data from Cycle 3 onward, this is
okay since the $T_{sys}$ and science data share the same
spectral window. &nbsp; Alternatively, the mapping between the $T_{sys}$ and
science spectral windows can be explicitly set with&nbsp;*spwmap*
and&nbsp;*spw.* In this case, we would use:

sdcal(infile=vis, calmode='ps,tsys,apply',&nbsp;spwmap={17:\[17\],
19:\[19\], 21:\[21\],23:\[23\]}, spw='17,19,21,23')

The general structure of *spwmap* is {Tsys spw 0: \[science spw
0\],....,Tsys spw n: \[science spw n\]} for 0 to n spectral windows.

**gencal** applied at this stage builds (and optionally applies) the
$T_{sys}$
calibration tables. These calibrations are an intrinsic part of the
ASDM. There are no re-computations applied to the&nbsp;$T_{sys}$ data by
CASA. Ultimately, the&nbsp;$T_{sys}$ calibration tables will be
applied in the **applycal** step, consistent with the descriptions of
calibrations given in the sections above. We point out that
the&nbsp;$T_{sys}$ calibrations are a
multiplicative factor, so the order of the application of the&nbsp;$T_{sys}$ cal
tables relative to the application of the&nbsp;$T_{sky}$
calibrations is immaterial.

**8. Subtracting the baseline: *sdbaseline***

It's important at this point to define exactly what is meant by
*baseline* in the context of single-dish data. In interferometry,
*baseline* refers to the spatial separation of antenna pairs.&nbsp; For
a single dish observation, *baseline* refers to the spectral pattern
produced by the atmosphere and instrument. Since single-dish antennas
measure total power, not an interference pattern, they are responsive to
emission wherever it exists within the single-dish beam or signal path.
This signal is dominated by the receiver/correlator/backend sampling
function, but has a significant time-varying component usually dominated
by atmospheric fluctuations. The power yielded by atmospheric
fluctuations are invisible to interferometer observations, as they are
in the *near field*, and are therefore generally resolved out from the
data.&nbsp; Note, though, that the atmospheric variability can
contaminate interferometric measurements by introducing a decoherence in
phase, and such losses in phase are not relevant for single-dish
observations.

**sdbaseline** removes a spectral baseline from the data on a
per-integration basis. The options here are extensive, and baseline
subtraction can be complex when emission is strongly variable throughout
the map, or when there are nearby atmospheric absorption features.&nbsp;
But CASA is effective at choosing intelligent defaults with
*mode='auto'*. With *mode='auto'*, CASA will examine the brightness
variability per integration and determine the most appropriate channel
ranges for computing the spectral baseline, based on the mean absolute
deviation of the channels. This approach is successful even when applied
to spectra crowded heavily with emission lines.&nbsp; As long as the
emission-free parts of the spectrum have statistically significant
representation in the data, then the *mode='auto'* will be successful.
Baseline corrections employed by CASA are subtracted, and therefore can
be applied iteratively, as needed.

**sdbaseline** supports Polynomial, Chebychev and Sinusoid baseline
removal. Sinusoidal baselines are determined with a Fourier transform of
the spectral data - again, an automatic mode is available, where CASA
will determine the most significant Fourier components and remove them,
though specific wavenumbers can be explicitly added or removed on top of
the automatic operation. Sinusoidal components occur in many single-dish
telescopes, and are a typical manifestation of a standing-wave
resonation of the main-reflector/subreflector cavity. ALMA has employed
scattering cones in the single-dish subreflectors to effectively
mitigate the strength of this standing wave. It's worth noting that
removal of Fourier components should be applied with utmost caution; the
result is effectively a convolution of the spectra with a spectral
filter, and MUST affect the resulting emission spectra. Users who use
this baseline mode should explore and characterize the consequences and
subsquent error propagation, in the context of their own data.  
  
In this example, we remove a 1st order polynomial from spectral windows
17, 19, 21 & 23, automatically finding and masking out any lines
brighter than 5 $\sigma$, and referencing the "corrected"
(i.e. calibrated) data column.

sdbaseline(infile = sd_ms,&nbsp;datacolumn = 'corrected',&nbsp;spw =
'17,19,21,23',&nbsp;maskmode = 'auto',&nbsp;thresh = 5.0,&nbsp;avg_limit
= 4,&nbsp;blfunc = 'poly',&nbsp;order = 1,&nbsp;outfile = sd_ms+'.cal')

&nbsp;

Note that at this point, the product dataset (sd_ms+'.cal) has only four
spectral windows. These are (if all is going well) the science
observations which are&nbsp;$T_{sys}$ and sky calibrated, and are now
bandpass-corrected.

&nbsp;

**9. Convert the Science Target Units from Kelvin to Jansky:
*scaleAutocorr***

To convert the units of the single-dish observations from $T_A^*$ (K) into
Janskys and to prepare for combination with interferometer data, we need
to obtain the empirically-determined Jy-to-K conversion data. These data
already take into account any correlator non-linearities and also factor
in the various subsystem efficiencies.&nbsp;

The easiest way to obtain this is simply with a call to a specialized
CASA task that obtains the Jy-to-K factors that accesses polynomal fits
from ongoing calibration campaign data.

jyperk = es.getJyPerK(sd_ms+'.cal')

The contents of this variable jyperk is a
python dictionary:

for ant in jyperk.keys():  
for spw in jyperk\[ant\].keys():  
scaleAutocorr(vis=sd_ms+'.cal', scale=jyperk\[ant\]\[spw\]\['mean'\],
antenna=ant, spw=spw)

&nbsp;

**scaleAutocorr** simply applies the scaling from $T_A^*$ to
Jy/beam. The scaling factors are determined empirically, as part of the
QA2-level calibrations provided by ALMA. The scaling factors are to be
provided to **scaleAutocorr** as a float, but are most conveniently
applied in calls that iterate through antenna and spectral window, where
the Jy-per-K factors are retained as a list with the format:

&nbsp;

&nbsp;jyperk = \\

&nbsp; { antenna01_name { spw0: { 'mean': 44.345, 'n': '', 'std': ''},

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; spw1: { 'mean': 44.374,
'n': '', 'std': ''},

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; spw2: { 'mean': 44.227,
'n': '', 'std': ''},

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; spw3: { 'mean': 44.203,
'n': '', 'std': ''}},

&nbsp; &nbsp; antenna02_name: { spw0: { 'mean': 44.345, 'n': '', 'std':
''},

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; spw1: { 'mean': 44.374,
'n': '', 'std': ''},

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; spw2: { 'mean': 44.227,
'n': '', 'std': ''},

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; spw3: { 'mean': 44.203,
'n': '', 'std': ''}}}

&nbsp;

which can be iterated and applied to the actual data with the following
loop:

to_amp_factor = lambda x: 1. / sqrt(x)

&nbsp;

for ant in jyperk.keys():

&nbsp; &nbsp;factors=\[\]

&nbsp; &nbsp;for spw in jyperk\[ant\].keys():

&nbsp; &nbsp; &nbsp; factors.append(jyperk\[ant\]\[spw\]\['mean'\])

&nbsp; &nbsp;gencal(vis=sd_ms, caltable=sd_ms+'.jy2ktbl', caltype='amp',
spw=",".join(str(x) for x in jyperk\[ant\].keys()),
parameter=map(to_amp_factor, factors))

&nbsp; &nbsp;applycal(vis=sd_ms+'.cal', gaintable=sd_ms+'.jy2ktbl')

&nbsp;

&nbsp;